# Project: SVD using Fixed Point Iterations

## Group members
* Jake Swartwout: Trying to break the system
* Lara Chunko
* Yunhan Yang

## Introduction

We looked at a sample way of finding the SVD for a matrix. But, the method seems highly unstable and slow, so we thought we would compare it to standard packages and attempt to break it.

The link to the repository: https://github.com/j2kun/svd/blob/master/svd.py

At a high level, this software performs the "Singular Value Decomposition" (SVD) of a matrix. This means that it is splitting any matrix A (it doesn't even need to be square) into 3 separate matrices. We represent it like this:

$$
A = U \Sigma V^T
$$

With the following properties:

$U$ and $V$ have orthonormal columns

$\Sigma$ is a diagonal matrix

This may seem familiar to diagonalization, as it does something similar. One of the main benefits however, is that our matrix A does not have to be square for SVD

In [1]:
from svdMaster.svd import svd as testSVD
from numpy.linalg import svd as npSVD
from scipy.linalg import svd as scipySVD

import numpy as np

import time

## Comparing ours to standard software

Comparing the runtimes of our SVD, numpy's svd, and Scipy's svd

In [10]:
#test array
A = np.random.rand(10,10) 

#applying testSVD
start_t = time.time()
test_sigma, test_U, test_VT = testSVD(A)
end_t = time.time()
test_time = end_t - start_t
print("testSVD runtime: ", test_time)
test_recombined = test_U @ np.diag(test_sigma) @ test_VT

#applying scipySVD 
start_t = time.time()
scipy_U, scipy_sigma, scipy_VT = scipySVD(A) 
end_t = time.time()
scipy_time = end_t - start_t
print("scipySVD runtime: ", scipy_time)
scipy_recombined = scipy_U @ np.diag(scipy_sigma) @ scipy_VT

#applying npSVD
start_t = time.time()
np_U, np_sigma, np_VT = npSVD(A)
end_t = time.time()
np_time = end_t - start_t
print("npSVD runtime: ", scipy_time)
np_recombined = np_U @ np.diag(np_sigma) @ np_VT

testSVD runtime:  0.0077245235443115234
scipySVD runtime:  0.0046651363372802734
npSVD runtime:  0.0046651363372802734


Comparing the recombined matrices among our software and the standard software

In [4]:
#testing testSVD versus scipySVD
#loop through and find the difference between the two versions
test_scipy_recombined_difference = 0
#column
for i in range(len(test_recombined[0])):
    #row
    for j in range(len(test_recombined)):
        #print("test ", test_recombined[i,j])
        #print("scipy ", scipy_recombined[i,j])
        test_scipy_recombined_difference += abs(test_recombined[i,j] - scipy_recombined[i,j])
        
print("test_scipy_recombined_difference = ", test_scipy_recombined_difference)

#testing testSVD versus npSVD
#loop through and find the difference between the two versions
test_np_recombined_difference = 0
#column
for i in range(len(test_recombined[0])):
    #row
    for j in range(len(test_recombined)):
        #print("test ", test_recombined[i,j])
        #print("np ", np_recombined[i,j])
        test_np_recombined_difference += abs(test_recombined[i,j] - np_recombined[i,j])
        
print("test_np_recombined_difference = ", test_np_recombined_difference)

#testing scipySVD versus npSVD
#loop through and find the difference between the two versions
scipy_np_recombined_difference = 0
#column
for i in range(len(scipy_recombined[0])):
    #row
    for j in range(len(scipy_recombined)):
        #print("scipy ", scipy_recombined[i,j])
        #print("np ", np_recombined[i,j])
        scipy_np_recombined_difference += abs(scipy_recombined[i,j] - np_recombined[i,j])

print("scipy_np_recombined_difference = ", scipy_np_recombined_difference)

test_scipy_recombined_difference =  3.933728343064047e-13
test_np_recombined_difference =  3.933728343064047e-13
scipy_np_recombined_difference =  0.0


We talk more about these results in the conclusion at the bottom

## Breaking it

This method does not seem very robust, so we decided to look into a few ways to break it.

The first step is to run several tests to get some averages

In [12]:
def split_combine_test(matrix):
    sigma, u, v = testSVD(A)
    recombined = u @ np.diag(sigma) @ v
    return recombined

In [22]:
numTests = 100
asize = (10,20)

In [13]:
norms = []
for i in range(numTests):
    A = np.random.rand(*asize)
    Aprime = split_combine_test(A)
    norms.append(np.linalg.norm(Aprime - A))
baseline = np.mean(norms)
print("Baseline reconstruction error for a random", asize, "matrix:", baseline)

Baseline reconstruction error for a random (10, 20) matrix: 4.531714141673653e-15


1. My first idea is to have two eigenvalues of the same value. This would mean that the program would not be able to choose one.

In [14]:
norms = []
for i in range(numTests):
    #generate a random matrix
    A = np.random.rand(*asize)
    #decompose it
    u, s, vt = npSVD(A, full_matrices=False)
    #scale the first two
    avg = (s[0] + s[1])/2
    s[0] = avg
    s[1] = avg
    #reconstruct it
    A = u @ np.diag(s) @ vt
    #split and recombine
    Aprime = split_combine_test(A)
    norms.append(np.linalg.norm(Aprime - A))
same = np.mean(norms)
print("Error with two same eigenvalues:", same)
print("Two same error : baseline error =", same / baseline, ": 1")

Error with two same eigenvalues: 3.0638269040502948e-15
Two same error : baseline error = 0.6760856506537638 : 1


2. Another idea would be to have all of them be the same length

In [15]:
norms = []
for i in range(numTests):
    #generate a random matrix
    A = np.random.rand(*asize)
    #decompose it
    u, s, vt = npSVD(A, full_matrices=False)
    #average all of them
    avg = np.mean(s)
    s = [avg for item in s]
    #reconstruct it
    A = u @ np.diag(s) @ vt
    #split and recombine
    Aprime = split_combine_test(A)
    norms.append(np.linalg.norm(Aprime - A))
all_same = np.mean(norms)
print("Error with two same eigenvalues:", all_same)
print("Two same error : baseline error =", all_same / baseline, ": 1")

Error with two same eigenvalues: 1.3968471710286546e-15
Two same error : baseline error = 0.30823814727924803 : 1


3. Or, the matrix could have all of it's columns be of length 1

In [19]:
norms = []
for i in range(numTests):
    #generate a random matrix
    A = np.random.rand(*asize)
    #scale everything
    for i in range(len(A[0])):
        A[:, i] /= np.linalg.norm(A[:, 0])
    #split and recombine
    Aprime = split_combine_test(A)
    norms.append(np.linalg.norm(Aprime - A))
len1 = np.mean(norms)
print("Error with two same eigenvalues:", len1)
print("Two same error : baseline error =", len1 / baseline, ": 1")

Error with two same eigenvalues: 4.293547837676208e-15
Two same error : baseline error = 0.9474445438190227 : 1


4. Or a matrix where two columns are just scaled versions of each other

In [21]:
norms = []
for i in range(numTests):
    #generate a random matrix
    A = np.random.rand(*asize)
    A[:, 1] = A[:, 0] * np.random.rand()
    #split and recombine
    Aprime = split_combine_test(A)
    norms.append(np.linalg.norm(Aprime - A))
scaled = np.mean(norms)
print("Error with two same eigenvalues:", scaled)
print("Two same error : baseline error =", scaled / baseline, ": 1")

Error with two same eigenvalues: 4.579761150626306e-15
Two same error : baseline error = 1.0106023918214109 : 1


## Conclusion

We noticed that scipy and numpy produce the same results, indicating that they use the same method to get their answer.

Consistently, our software runs slower than the scipy/numpy software. So, we would not recommend using this software in a professional setting, as it would take much longer to get your solution.

Surprisingly, all of these failed to break our system, meaning that it is likely pretty robust. Perhaps this is due to overall low accuracy to begin with, so small errors don't change it much. I would be interested in knowing why the error actually goes down for some of the methods, as I would estimate that it should be getting worse.